In [1]:
from pipeline.pipeline import Pipeline
from music21 import converter, midi, key, instrument, meter, interval, note, pitch, stream, tempo, chord
import os
import pandas as pd
import mido
import glob

import pretty_midi

from tqdm import tqdm

import mido
import pandas as pd

from utils.utils import open_midi, get_file_name, extract_tempo, extract_key, to_snake_case

In [2]:
p_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/program_change.csv')
com_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_addedpitch.csv', index_col=0)
# df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre.csv')

In [3]:
m_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/movie.csv')
t_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/tv.csv')
v_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/videogame.csv')

m_df.sort_values(by=['song_midi', 'file_name'], inplace=True)
t_df.sort_values(by=['song_midi', 'file_name'], inplace=True)
v_df.sort_values(by=['song_midi', 'file_name'], inplace=True)

m_df.reset_index(drop=True, inplace=True)
t_df.reset_index(drop=True, inplace=True)
v_df.reset_index(drop=True, inplace=True)

m_df['genre'] = 'movie'
t_df['genre'] = 'tv'
v_df['genre'] = 'videogame'

percs_num = [i for i in range(112, 128)]
percs_num.append(47)
# df = pd.concat([m_df, t_df, v_df], ignore_index=True).reset_index(drop=True)
# # df.to_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre.csv', index=False)

In [4]:
# def drop_perc(df, percs_num):
#     for idx in range(len(df)):
#         if df['program_change_value'][idx] in percs_num:
#             df.drop(idx, inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     return df

In [5]:
commu_folder = './commu_midi'
com_m = glob.glob(f'{commu_folder}/*.mid')
com_m.sort()

genre_list = ['movie' , 'tv', 'videogame']
df_list = [m_df, t_df, v_df]

In [16]:
def get_pitch_range(df_list, genre_list):

    new_df = pd.DataFrame()

    for i in range(len(df_list)):
        df = df_list[i]
        genre = genre_list[i]

        data_folder = './data/' + genre + '/cropped'
        folder_list = set(df['song_midi'].to_list())

        our_m = list()
        for idx, v in enumerate(df['song_midi'].values):
            our_m.append(os.path.join(data_folder, v + '/' + df['file_name'].values[idx]))
        
        df['min_pitch'] = 0
        df['max_pitch'] = 0
        df['mean_pitch'] = 0

        df['min_velocity'] = 0
        df['max_velocity'] = 0
        df['chord_progression'] = None

        for idx, midi in tqdm(enumerate(our_m)):
            try:
                score = converter.parse(midi)
            except:
                print(f'Error: {idx}')
                continue

            m_numbers = list()
            v_numbers = list()
            names = list()

            for obj in score.recurse():
                if isinstance(obj, note.Note):
                    m_numbers.append(obj.pitch.midi)
                    v_numbers.append(obj.volume.velocity)
                    names.append(obj.nameWithOctave)
                elif isinstance(obj, chord.Chord):
                    names.append(obj.pitchedCommonName)
                    for p in obj.pitches:
                        m_numbers.append(p.midi)
                        v_numbers.append(obj.volume.velocity)
            
            m_numbers = [i for i in m_numbers if i >= 19]

            if len(m_numbers) == 0:
                print(f'No notes: {idx}')
                continue
            
            df.loc[idx, 'min_pitch'] = min(m_numbers)
            df.loc[idx, 'max_pitch'] = max(m_numbers)
            df.loc[idx, 'mean_pitch'] = sum(m_numbers) / len(m_numbers)

            df.loc[idx, 'min_velocity'] = min(v_numbers)
            df.loc[idx, 'max_velocity'] = max(v_numbers)
            df.at[idx, 'chord_progression'] = names
        
        new_df = pd.concat([new_df, df], ignore_index=True)
        
    return new_df

In [17]:
new_df = get_pitch_range(df_list, genre_list)

27270it [08:25, 53.92it/s]
14813it [04:07, 59.81it/s]
18318it [06:05, 50.14it/s] 


In [6]:
# com_df['min_pitch'] = 0
# com_df['max_pitch'] = 0
# com_df['mean_pitch'] = 0

# for idx, midi in enumerate(com_m):
#     score = converter.parse(midi)
#     m_numbers = list()
#     print(idx)
#     for obj in score.recurse():
#         if isinstance(obj, note.Note):
#             m_numbers.append(obj.pitch.midi)
#         elif isinstance(obj, chord.Chord):
#             for p in obj.pitches:
#                 m_numbers.append(p.midi)
#     com_df.loc[idx, 'min_pitch'] = min(m_numbers)
#     com_df.loc[idx, 'max_pitch'] = max(m_numbers)
#     com_df.loc[idx, 'mean_pitch'] = sum(m_numbers) / len(m_numbers)

In [18]:
com_df.groupby('pitch_range').mean().apply(lambda x: round(x, 0))

,num_measures,bpm,min_velocity,max_velocity,min_pitch,max_pitch,mean_pitch
pitch_range,,,,,,,
high,7.0,88.0,52.0,63.0,79.0,95.0,87.0
low,8.0,94.0,70.0,78.0,36.0,48.0,42.0
mid,8.0,92.0,62.0,74.0,55.0,75.0,66.0
mid_high,8.0,92.0,58.0,70.0,68.0,86.0,77.0
mid_low,8.0,92.0,59.0,70.0,43.0,65.0,56.0
very_high,4.0,70.0,52.0,66.0,90.0,104.0,98.0
very_low,8.0,91.0,74.0,81.0,28.0,36.0,32.0


In [19]:
new_df['mean_pitch'] = new_df['mean_pitch'].apply(lambda x: int(x))

In [20]:
very_high = [i for i in range(95, 127)]
high = [i for i in range(86, 95)]
mid_high = [i for i in range(75, 86)]
mid = [i for i in range(65, 75)]
mid_low = [i for i in range(48, 65)]
low = [i for i in range(36, 48)]
very_low = [i for i in range(0, 36)]

In [22]:
new_df['pitch_range'] = None

for idx, v in enumerate(new_df['mean_pitch']):
    current_value = round(v, 0)

    if current_value in very_high:
        new_df.loc[idx, 'pitch_range'] = 'very_high'
    elif current_value in high:
        new_df.loc[idx, 'pitch_range'] = 'high'
    elif current_value in mid_high:
        new_df.loc[idx, 'pitch_range'] = 'mid_high'
    elif current_value in mid:
        new_df.loc[idx, 'pitch_range'] = 'mid'
    elif current_value in mid_low:
        new_df.loc[idx, 'pitch_range'] = 'mid_low'
    elif current_value in low:
        new_df.loc[idx, 'pitch_range'] = 'low'
    elif current_value in very_low:
        new_df.loc[idx, 'pitch_range'] = 'very_low'

In [23]:
# new_df.to_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_p.csv', index=False)

In [58]:
# for idx, v in enumerate(df['chord_progression']):
#     current_list = list()
#     for i in v:
#         current_list.append(to_snake_case(i))
#     df.at[idx, 'chord_progression'] = current_list

In [62]:
df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_p.csv')

In [64]:
com_df

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,min_pitch,max_pitch,mean_pitch
0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,57,81,69.055556
1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,41,67,58.666667
2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,71,89,81.140625
3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,62,76,68.925926
4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,40,60,50.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140,60,91,73.800000
11140,cmajor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141,53,67,59.250000
11141,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142,33,33,33.000000
11142,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143,40,48,43.375000


In [66]:
com_df['chord_progressions'] = None

In [68]:
for idx, midi in tqdm(enumerate(com_m)):
    try:
        score = converter.parse(midi)
    except:
        print(f'Error: {idx}')
        continue

    names = list()

    for obj in score.recurse():
        if isinstance(obj, note.Note):
            names.append(obj.nameWithOctave)
        elif isinstance(obj, chord.Chord):
            names.append(obj.pitchedCommonName)
    
    com_df.at[idx, 'chord_progressions'] = names

11144it [00:44, 250.10it/s]


In [69]:
com_df

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,min_pitch,max_pitch,mean_pitch
0,aminor,"[Perfect Octave above A, Perfect Octave above ...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,57,81,69.055556
1,cmajor,"[C3, G3, Major Third above C, G3, G3, Major Th...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,41,67,58.666667
2,aminor,"[A5, C5, E5, A5, C6, E5, A5, C6, E6, A5, C6, E...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,71,89,81.140625
3,cmajor,"[C-major triad, G-minor triad, C-major triad, ...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,62,76,68.925926
4,aminor,"[A-minor triad, E-minor triad, F-major triad, ...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,40,60,50.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"[Major Seventh with octave doublings above F, ...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140,60,91,73.800000
11140,cmajor,"[A3, F3, C4, G4]",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141,53,67,59.250000
11141,aminor,"[A1, A1]",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142,33,33,33.000000
11142,aminor,"[Perfect Fourth above E, Perfect Fourth above ...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143,40,48,43.375000


In [71]:
for idx, v in enumerate(com_df['chord_progressions']):
    current_list = list()
    for i in v:
        current_list.append(to_snake_case(i))
    com_df.at[idx, 'chord_progressions'] = current_list

In [73]:
com_df.to_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_cp.csv', index=False)